# MPS states and their properties
Matrix product states (MPS) are states in which 
\begin{equation}
c_{i_0,i_1\cdots i_{N-2},i_{N-1}} = \textrm{tTr}\left(A^{i_1}A^{i_2}\cdots A^{i_{N-2}}A^{i_{N-1}}\right)
\end{equation} today we will study some of their properties and see how they help computing things. 

Let's start by trying to obtain a MPS for the random states we generated in last tutorials follwing the recipe that we have seen in books that is by performing iterated SVDs, 
![MPS_construction](../pictures/MPS.png "from doi:10.1088/1751-8121/aa6dc3")

Start with $N=4$


In [23]:
import numpy as np 
import scipy.linalg as LA
def generate_random_state(N):
        dim_h =2**N
        init_state = np.zeros([dim_h,1])
        init_state[0]=1.
        random_h = np.array(np.random.rand(dim_h,dim_h)+1j*np.random.rand(dim_h,dim_h))
        random_h = random_h+random_h.T.conj()
        random_h = random_h/LA.norm(random_h)*N

        random_unitary =LA.expm(-1j*random_h)
        random_state=random_unitary@init_state
        return random_state
    
def compute_A_sigma_bulk(remaining_state,n):
        remaining_spins= len(remaining_state.shape)
        if n == 0 :
            print('first')
            first_size =remaining_state.shape[0]
            second_size=np.prod(remaining_state.shape[1:])
        else:
            print('bulk')
            first_size =np.prod(remaining_state.shape[0:2])
            second_size=np.prod(remaining_state.shape[2:])
            
            
        remaining_state_matrix = remaining_state.reshape(
            first_size,second_size)
        A,sigma,rest = LA.svd(remaining_state_matrix,full_matrices
                             =False)
        
        chi=rest.shape[0]
        sigma =np.diag(sigma)
        if n == 0:
            rest_tensor =rest.reshape([rest.shape[0]]
                                  +list(remaining_state.shape[1:]))
        else:
            A =A.reshape(remaining_state.shape[0],remaining_state.shape[1],
                     A.shape[1])
        
            rest_tensor =rest.reshape([rest.shape[0]]+
                                  list(remaining_state.shape[2:]))
        
        return A, sigma, rest_tensor
        
def state_reconstruct_einsum(list_A,list_sigma):
    N =len(list_A)
    avail_index = 'abcdefghlmnopqrstuvz'
    first_piece = list_A_tensors[0]
    for n in range(N-1):
        if n ==0:
            first_piece=np.einsum('ab,bd->ad',first_piece,list_sigma_tensors[n])
        else: 
            #print(first_piece.shape)
            #print(list_sigma_tensors[n].shape)
            num_left_indices=len(first_piece.shape)
            einsum_index=avail_index[0:num_left_indices]
            einsum_index+=','+avail_index[num_left_indices-1:num_left_indices+1]
            einsum_index+='->'+avail_index[0:num_left_indices-1]+avail_index[
               num_left_indices]
            #print(einsum_index)
            first_piece=np.einsum(einsum_index,
                                      first_piece,list_sigma_tensors[n])
        print('contracting '+str(n))
        num_left_indices=len(first_piece.shape)
        einsum_index=avail_index[0:num_left_indices]
        next_piece = list_A_tensors[n+1]
        other_index=len(next_piece.shape)
        einsum_index = einsum_index+','+avail_index[num_left_indices-1:
                    num_left_indices+other_index-1]
        einsum_index += '->'+avail_index[0:num_left_indices-1]+ avail_index[
            num_left_indices:num_left_indices+other_index-1]
        #print(einsum_index)
        first_piece =np.einsum(einsum_index,first_piece,next_piece)    
    return first_piece
def state_reconstruct_matmult(list_A,list_sigma):
    N =len(list_A)
    #print(N)
    list_d =[]
    list_d.append(list_A[0].shape[1])
    mat_left= list_A[0]
    for n in range(1,N-1):
        #print(n)
        mat_left = mat_left@list_sigma[n-1]
        mat_left = mat_left@list_A[n].reshape(list_A[n].shape[0],list_A[n].shape[1]*list_A[n].shape[2])
        mat_left = mat_left.reshape(mat_left.shape[0]*list_A[n].shape[1],list_A[n].shape[2])
        list_d.append(list_A[n].shape[1])
    
    mat_left = mat_left@list_sigma[N-2]
    mat_left = mat_left@list_A[N-1]
    list_d.append(list_A[N-1].shape[1])
    mat_left = mat_left.reshape(list_d)
    return mat_left
        
N=6
rand_state=generate_random_state(N);
tensor_dim =[2]*N
rand_state_tensor = rand_state.reshape(tensor_dim)
remaining_state=rand_state_tensor
list_A_tensors =[]
list_sigma_tensors =[]
for n in range(N-1):
    #print(n)
    A,sigma,rest=compute_A_sigma_bulk(remaining_state,n)
    
    remaining_state =rest
    #print(rest.shape)
    list_A_tensors.append(A)
    list_sigma_tensors.append(sigma)

list_A_tensors.append(rest)    
recontructed_state_einsum = state_reconstruct_einsum(list_A_tensors,list_sigma_tensors)
recontructed_state_mat_mult = state_reconstruct_matmult(list_A_tensors,list_sigma_tensors)
print(np.max(rand_state_tensor-recontructed_state_einsum))
print(np.max(rand_state_tensor-recontructed_state_mat_mult))



first
bulk
bulk
bulk
bulk
contracting 0
contracting 1
contracting 2
contracting 3
contracting 4
(1.8041124150158794e-16-4.163336342344337e-17j)
(1.9081958235744878e-16-2.0816681711721685e-17j)
